In [ ]:
# Loading libraries
import subprocess
import csv

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.utils import resample

from skopt import BayesSearchCV

from scipy import stats
from scipy.stats import mannwhitneyu
from statsmodels.stats import multitest

import seaborn as sns

In [ ]:
# Age prediction data processing
age_data_expr=pd.read_table('./data/progeria.healthy.tmm.tsv', index_col=None).T
age_data_expr['Run']=age_data_expr.index
age_data_expr['Run']=age_data_expr['Run'].apply( lambda x: x.replace('_quant', ''))

age_metadata=pd.read_table('./data/progeria.metadata')

In [ ]:
# Merge metadata info with expressions
age_data=pd.merge(age_data_expr, age_metadata[['Age', 'Run']], on='Run').drop(columns='Run')
age_data['Age']=age_data['Age'].astype('int')
age_data=age_data.loc[age_data['Age']>=20, :]
age_data_num=age_data

In [ ]:
# Creating age categorical variable

# 20-30: 0
# 31-44: 1
# 45-59: 2
# 60-74: 3
# 75+  : 4

age_data['Age_cat']=np.where(age_data['Age']<31, "0",
                             np.where(age_data['Age']<45, "1",
                                      np.where(age_data['Age']<60, "2",
                                               np.where(age_data['Age']<75, "3", "4"))))
age_data=age_data.drop(columns='Age')

In [ ]:
# Writing files for filter
exprs="data/exprs.txt"
conds="data/conds.txt"

age_data.drop(columns=["Age_cat"]).to_csv(exprs, sep="\t", index=False, header=False, decimal=",")
age_data['Age_cat'].to_csv(conds, sep="\t", index=False, header=False, quoting=csv.QUOTE_ALL)

In [ ]:
# Running filter
RauLCF="./RauLCF/RauLCF_UI.exe -m " + exprs + " -v " + conds + " -i 1 200 -c 25"
res=subprocess.run(RauLCF, capture_output=True, text=True)

In [ ]:
# Removing all genes below threshold
threshold=float(res.stdout[:-1].replace(",", "."))

expr_cols=age_data.drop(columns=['Age_cat']).columns

for col in expr_cols:
    if (age_data[col].sum() < threshold*age_data[col].count()) & (age_data[col].max() < threshold):
        age_data.drop(col, axis=1, inplace=True)

In [ ]:
age_data_train=age_data

In [ ]:
age_data_train["Age_cat"].value_counts()

In [ ]:
# Function which returns feature importance on 80% observations
def get_features_stability(data, clf, rand_state):
    sample=resample(data, n_samples=int(data.shape[0]*0.8), stratify=data.iloc[:, -1], random_state=rand_state)
    clf=clf.fit(sample.iloc[:,:-1], sample.iloc[:, -1].astype('int'))
    feature_importance=pd.DataFrame(clf.feature_importances_, sample.iloc[:,:-1].columns)
    feature_importance.rename(columns={0:"Importance"}, inplace=True)
    return(feature_importance)

In [ ]:
# Running Bayesian search with xgboost
XGBclf=BayesSearchCV(
    xgb.XGBClassifier(objective="multi:softmax", num_class="4", random_state=500),
    {
        'n_estimators': (5, 500),
        'max_depth' : (2, 500),
        'max_leaves' : (2, 4),
        'learning_rate' : (0.0001, 0.9),
        'booster' : ("gbtree", "gblinear", "dart"),
        'reg_alpha' : (0.0001, 1)
    },
    cv=2,
    n_jobs=24,
    random_state=500
)
XGBclf.fit(age_data_train.iloc[:,:-1], age_data_train.iloc[:, -1].astype('int'))
best_params_xgb=XGBclf.best_params_

print(best_params_xgb)

In [ ]:
# Defining XGB with the best hyperparameters
XGBclf_best=xgb.XGBClassifier(n_estimators=best_params_xgb['n_estimators'],
                              max_depth=best_params_xgb['max_depth'],
                              max_leaves=best_params_xgb['max_leaves'],
                              learning_rate=best_params_xgb['learning_rate'],
                              booster=best_params_xgb['booster'],
                              reg_alpha=best_params_xgb['reg_alpha'],
                              objective="multi:softmax",
                              num_class="4",
                              random_state=500)

# Obtaining feature importance for different data subsets
for i in range(100):
    importance=get_features_stability(age_data_train, XGBclf_best, i).abs().mean(axis=1).sort_values(ascending=False)
    if i==0:
        stability_xgb=importance.iloc[:50]
    else:
        stability_xgb=pd.concat([stability_xgb, importance.iloc[:50]], axis=1)

In [ ]:
stability_xgb[stability_xgb.isna().sum(axis=1) <= 50]

In [ ]:
stability_xgb['genes']=stability_xgb.index
stability_xgb.loc[stability_xgb.isna().sum(axis=1) <= 50, 'genes'].to_csv('importance_stability_XGB_tmm.txt', sep="\t", index=False, header=False)

In [ ]:
# Running Bayesian search with Logistic Regression
LogRclf=BayesSearchCV(
    LogisticRegression(penalty="elasticnet", solver="saga", random_state=500),
    {
        'C': (0.0001, 2),
        'max_iter' : (50, 500),
        'l1_ratio' : (0, 1)
    },
    cv=2,
    n_jobs=24,
    random_state=500
)
LogRclf.fit(age_data_train.iloc[:,:-1], age_data_train.iloc[:, -1].astype('int'))
best_params_logr=LogRclf.best_params_

print(best_params_logr)

In [ ]:
# Defaining Logistic Regression with the best hyperparameters
LogRclf_best=LogisticRegression(penalty="elasticnet",
                                solver="saga",
                                C=best_params_logr['C'],
                                max_iter=best_params_logr['max_iter'],
                                l1_ratio=best_params_logr['l1_ratio'],
                                random_state=500)

# Obtaining feature importance for different data subsets
for i in range(100):
    sample=resample(age_data_train, n_samples=int(age_data_train.shape[0]*0.8), stratify=age_data_train.iloc[:, -1], random_state=i)
    LogRclf_best=LogRclf_best.fit(sample.iloc[:,:-1], sample.iloc[:, -1].astype('int'))
    importance=pd.DataFrame(LogRclf_best.coef_.T, sample.iloc[:,:-1].columns).abs().mean(axis=1).sort_values(ascending=False)
    if i==0:
        stability_logr=importance.iloc[:50]
    else:
        stability_logr=pd.concat([stability_logr, importance.iloc[:50]], axis=1)

In [ ]:
stability_logr[stability_logr.isna().sum(axis=1) <= 50]

In [ ]:
stability_logr['genes']=stability_logr.index
stability_logr.loc[stability_logr.isna().sum(axis=1) <= 50, 'genes'].to_csv('importance_stability_LogR.txt', sep="\t", index=False, header=False)

In [ ]:
# Age groups

# 20-30: 0
# 31-44: 1
# 45-59: 2
# 60-74: 3
# 75+  : 4

def get_cruscal(genes):
    p_vals=[]
    for gene in genes:
        Age_20_30=age_data_train.loc[age_data_train['Age_cat']=="0", gene].to_list()
        Age_31_44=age_data_train.loc[age_data_train['Age_cat']=="1", gene].to_list()
        Age_45_59=age_data_train.loc[age_data_train['Age_cat']=="2", gene].to_list()
        Age_60_74=age_data_train.loc[age_data_train['Age_cat']=="3", gene].to_list()
        Age_75_more=age_data_train.loc[age_data_train['Age_cat']=="4", gene].to_list()

        stat, p_val=stats.kruskal(Age_20_30, Age_31_44, Age_45_59, Age_60_74, Age_75_more, nan_policy='omit')
        p_vals.append(p_val)
    rej, p_adj, alphsid, alphb=multitest.multipletests(p_vals, alpha=0.05, method='fdr_bh')

    return(pd.DataFrame(p_adj, genes))


In [ ]:
genes=pd.read_table("./data/Biom.txt", index_col=None, header=None)
get_cruscal(genes[0].values.tolist())# All genes with p-val<0.05 are genes of interest

In [ ]:
# Plot gene expr vs age and fit lowess curve
lowess_plt=sns.regplot(data=age_data_num, x="Age", y="CRIM1", lowess=True)

In [ ]:
# Test pairs per every gene of interest

gene_list=['SRF','SNHG32','MIR22HG','RPL3','PPDPF','ADPGK','ZNF146','TNC','TPM4','SAT1','CRIM1']

age_1=age_data_num.loc[age_data_num['Age']<35]
age_2=age_data_num.loc[(age_data_num['Age']>35) & (age_data_num['Age']<55)]
age_3=age_data_num.loc[(age_data_num['Age']>55) & (age_data_num['Age']<75)]
age_4=age_data_num.loc[age_data_num['Age']>75]

pairs_list={'SRF':[[age_1['SRF'], age_2['SRF']],
                   [age_2['SRF'], age_3['SRF']],
                   [age_1['SRF'], age_3['SRF']],
                   [age_2['SRF'], age_4['SRF']],
                   [age_3['SRF'], age_4['SRF']]],
            'SNHG32':[[age_1['SNHG32'], age_2['SNHG32']],
                      [age_1['SNHG32'], age_3['SNHG32']],
                      [age_1['SNHG32'], age_4['SNHG32']],
                      [age_3['SNHG32'], age_4['SNHG32']]],
            'MIR22HG':[[age_1['MIR22HG'], age_2['MIR22HG']],
                       [age_2['MIR22HG'], age_3['MIR22HG']],
                       [age_1['MIR22HG'], age_3['MIR22HG']],
                       [age_2['MIR22HG'], age_4['MIR22HG']],
                       [age_3['MIR22HG'], age_4['MIR22HG']]],
            'RPL3':[[age_1['RPL3'], age_2['RPL3']],
                    [age_2['RPL3'], age_3['RPL3']],
                    [age_1['RPL3'], age_4['RPL3']],
                    [age_2['RPL3'], age_4['RPL3']],
                    [age_3['RPL3'], age_4['RPL3']]],
            'PPDPF':[[age_1['PPDPF'], age_4['PPDPF']],
                     [age_2['PPDPF'], age_4['PPDPF']],
                     [age_3['PPDPF'], age_4['PPDPF']]],
            'ADPGK':[[age_1['ADPGK'], age_2['ADPGK']],
                     [age_1['ADPGK'], age_3['ADPGK']],
                     [age_2['ADPGK'], age_3['ADPGK']],
                     [age_1['ADPGK'], age_4['ADPGK']],
                     [age_2['ADPGK'], age_4['ADPGK']],
                     [age_3['ADPGK'], age_4['ADPGK']]],
            'ZNF146':[[age_1['ZNF146'], age_4['ZNF146']],
                      [age_2['ZNF146'], age_4['ZNF146']],
                      [age_3['ZNF146'], age_4['ZNF146']]],
            'TNC': [[age_1['TNC'], age_2['TNC']],
                    [age_1['TNC'], age_4['TNC']],
                    [age_2['TNC'], age_3['TNC']],
                    [age_2['TNC'], age_4['TNC']],
                    [age_3['TNC'], age_4['TNC']]],
            'TPM4':[[age_1['TPM4'], age_2['TPM4']],
                    [age_1['TPM4'], age_4['TPM4']],
                    [age_2['TPM4'], age_3['TPM4']],
                    [age_2['TPM4'], age_4['TPM4']],
                    [age_3['TPM4'], age_4['TPM4']]],
            'SAT1':[[age_1['SAT1'], age_2['SAT1']],
                    [age_1['SAT1'], age_3['SAT1']],
                    [age_2['SAT1'], age_3['SAT1']],
                    [age_1['SAT1'], age_4['SAT1']],
                    [age_2['SAT1'], age_4['SAT1']],
                    [age_3['SAT1'], age_4['SAT1']]],
            'CRIM1':[[age_1['CRIM1'], age_2['CRIM1']],
                     [age_1['CRIM1'], age_4['CRIM1']],
                     [age_2['CRIM1'], age_3['CRIM1']],
                     [age_2['CRIM1'], age_4['CRIM1']],
                     [age_3['CRIM1'], age_4['CRIM1']]]}

def get_utest(pairs, genes):
    p_vals=[]
    for gene in genes:
        for pair in pairs[gene]:
            stat, p_val = mannwhitneyu(pair[0], pair[1], nan_policy='omit')
            p_vals.append(p_val)

    return(p_vals)

In [ ]:
p_vals=get_utest(pairs_list, gene_list)
p_vals # all unadjasted p-vals

In [ ]:
rej, p_adj, alphsid, alphb=multitest.multipletests(p_vals, alpha=0.05, method='fdr_bh')
p_adj #All BH adjusted p-vals

In [ ]:
#DF for export
df_p_adj=pd.DataFrame({'p_adj':p_adj.tolist(),
                       'gene':['SRF','SRF','SRF','SRF','SRF','SNHG32','SNHG32','SNHG32','SNHG32',
                               'MIR22HG','MIR22HG','MIR22HG','MIR22HG','MIR22HG','RPL3','RPL3',
                               'RPL3','RPL3','RPL3','PPDPF','PPDPF','PPDPF','ADPGK','ADPGK',
                               'ADPGK','ADPGK','ADPGK','ADPGK','ZNF146','ZNF146','ZNF146','TNC',
                               'TNC','TNC','TNC','TNC','TPM4','TPM4','TPM4','TPM4','TPM4','SAT1',
                               'SAT1','SAT1','SAT1','SAT1','SAT1','CRIM1','CRIM1','CRIM1',
                               'CRIM1','CRIM1'],
                       'age_group':['1_vs_2', '2_vs_3', '1_vs_3', '2_vs_4', '3_vs_4', '1_vs_2',
                                    '1_vs_3', '1_vs_4', '3_vs_4', '1_vs_2', '2_vs_3', '1_vs_3', '2_vs_4', '3_vs_4', '1_vs_2', '2_vs_3', '1_vs_4', '2_vs_4', '3_vs_4', '1_vs_4', '2_vs_4', '3_vs_4', '1_vs_2', '1_vs_3', '2_vs_3', '1_vs_4', '2_vs_4', '3_vs_4', '1_vs_4', '2_vs_4', '3_vs_4', '1_vs_2', '1_vs_4', '2_vs_3', '2_vs_4', '3_vs_4', '1_vs_2', '1_vs_4', '2_vs_3', '2_vs_4', '3_vs_4', '1_vs_2', '1_vs_3', '2_vs_3', '1_vs_4', '2_vs_4', '3_vs_4', '1_vs_2', '1_vs_4', '2_vs_3', '2_vs_4', '3_vs_4']})

In [ ]:
#Save DF with significant differences of gene expr between defined groups
df_p_adj[df_p_adj['p_adj']<0.05].to_csv("age_groups_biomarkers.txt", sep="\t", index=False)

In [ ]:
#Save DF with all tested gene-age group pairs
df_p_adj.drop(columns='p_adj').to_csv("tests_design.txt", sep="\t", index=False)

In [ ]:
#Age groups desc stats
age_descr=age_data_num
age_descr['Age_cat']=np.where(age_descr['Age']<35, "1",
                              np.where(age_descr['Age']<55, "2",
                                       np.where(age_descr['Age']<75, "3", "4")))

age_descr.groupby(by='Age_cat')['Age'].describe()